In [58]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
# from dash import dcc
import dash_core_components as dcc
# from dash import html
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
# from dash import dash_table as dt
import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
# from bson.json_util import dumps
# import json
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalshelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = 'aacuser'
password = 'password'
shelter = AnimalShelter(username, password, 27017)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
checklist_options = ['Water Rescue', 'Mountain/Wilderness Rescue', 'Disaster/Individual Rescue']

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image = open('Grazioso Salvare Logo.png', 'rb')
encoded_image = base64.b64encode(image.read())
image.close()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Header('Jason Verrill')),
    html.Center(
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()), title='www.snhu.edu'),
                href='https://www.snhu.edu')),
    html.Hr(),
    html.Div([
        
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Checklist(id='checklist_id', options=checklist_options, value=[]),
        html.Button('Reset', id='reset_button', n_clicks=0),

    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                #id='graph-id',
                #className='col s12 m6',
                dcc.Graph(id='graph')
            ),
            html.Div(
                #id='map-id',
                #className='col s12 m6',
                #dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10)
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Checklist used to filter data table
# If nothing is checked, unfiltered data displays
@app.callback(
    Output(component_id='datatable-id', component_property='data'),
    Input(component_id='checklist_id', component_property='value'),
    prevent_initial_call=True
)
def filter_table(value):
    
    # Create dataframe with correct columns and empty rows
    dff = pd.DataFrame(columns=[i for i in df.columns])
    
    # If no option is checked, show all rows and return early
    if value == []:
        return df.to_dict('records')
    
    # Build filtered dataframes and concatenate together based on checklist values
    if 'Water Rescue' in value:
        # Filters
        lab = df[df['breed'].str.contains('Lab')]
        chesa = df[df['breed'].str.contains('Chesa')]
        newfound = df[df['breed'].str.contains('Newfound')]
        
        # Concatenate filters
        dff = pd.concat((dff, lab))
        dff = pd.concat((dff, chesa))
        dff = pd.concat((dff, newfound))
        
    if 'Mountain/Wilderness Rescue' in value:
        # Filters
        german = df[df['breed'].str.contains('German')]
        alaskan = df[df['breed'].str.contains('Alaskan Malamute')]
        english = df[df['breed'].str.contains('Old English')]
        sheepdog = df[df['breed'].str.contains('Sheepdog')]
        husky = df[df['breed'].str.contains('Siberian Husky')]
        rott = df[df['breed'].str.contains('Rottweiler')]
        
        # Concatenate filters
        dff = pd.concat((dff, german))
        dff = pd.concat((dff, alaskan))
        dff = pd.concat((dff, english))
        dff = pd.concat((dff, sheepdog))
        dff = pd.concat((dff, husky))
        dff = pd.concat((dff, rott))
        
    if 'Disaster/Individual Rescue' in value:
        # Filters
        dober = df[df['breed'].str.contains('Dober')]
        german = df[df['breed'].str.contains('German')]
        golden = df[df['breed'].str.contains('Golden')]
        bloodhound = df[df['breed'].str.contains('Bloodhound')]
        rott = df[df['breed'].str.contains('Rott')]
        
        # Concatenate filters
        dff = pd.concat((dff, dober))
        dff = pd.concat((dff, german))
        dff = pd.concat((dff, golden))
        dff = pd.concat((dff, bloodhound))
        dff = pd.concat((dff, rott))
        
    return dff.to_dict('records')


# Reset button
@app.callback(
    Output(component_id='checklist_id', component_property='value'),
    Input(component_id='reset_button', component_property='n_clicks'),
    prevent_initial_call=True
)
def reset_table(n_clicks):
# If reset is clicked, use original data with no filters
    if n_clicks > 0:
        n_clicks = 0
        return []


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph', 'figure'),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    
    table_df = pd.DataFrame(viewData)
    
    
    # Duplicates for rottweiler and german shepherd not listed

    # Water Rescue counts
    num_lab = len(table_df[table_df['breed'].str.contains('Lab')])
    num_chesa = len(table_df[table_df['breed'].str.contains('Chesa')])
    num_newfound = len(table_df[table_df['breed'].str.contains('Newfound')])
    
    # Mountain/Wilderness Rescue counts
    num_german = len(table_df[table_df['breed'].str.contains('German')])
    num_alaskan = len(table_df[table_df['breed'].str.contains('Alaskan Malamute')])
    num_english = len(table_df[table_df['breed'].str.contains('Old English')])
    num_sheepdog = len(table_df[table_df['breed'].str.contains('Sheepdog')])
    num_husky = len(table_df[table_df['breed'].str.contains('Siberian Husky')])
    num_rott = len(table_df[table_df['breed'].str.contains('Rottweiler')])
    
    # Rescue Disaster/Individual Rescue counts
    num_dober = len(table_df[table_df['breed'].str.contains('Dober')])
    num_golden = len(table_df[table_df['breed'].str.contains('Golden')])
    num_bloodhound = len(table_df[table_df['breed'].str.contains('Bloodhound')])

    
    # Names for the graph
    names = [
        'Labrador Retriever',
        'Chesapeake Bay Retriever',
        'Newfoundland',
        
        'German Shepherd',
        'Alaskan Malamute',
        'Old English Sheepdog',
        'Sheepdog',
        'Siberian Husky',
        'Rottweiler',
        
        'Doberman Pinscher',
        'Golden Retreiver',
        'Bloodhound'
    ]
    
    # Totals for the graph
    totals = [
        num_lab, num_chesa, num_newfound,
        num_german, num_alaskan, num_english, num_sheepdog, num_husky, num_rott,
        num_dober, num_golden, num_bloodhound]
    
    # Create graph
    fig = go.Figure(
    data=[go.Bar(x=names, y=totals)],
    layout_title_text='Rescue Dog Breeds'
    )
    
    return fig


#FIXME: Add in the code for your geolocation chart
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_viewport_data")
)
# Display geolocation map
def update_map(mapData):
    geo_data = pd.DataFrame.from_dict(mapData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(geo_data.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(geo_data.iloc[1,9])
                ])
            ])
        ])
    ]

app